In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import zipfile
zip_path = "/content/drive/MyDrive/ML/data.zip"
unzip_path = "/content/data"
if not os.path.exists(unzip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms, models
from torchvision.models import ResNet50_Weights
from tqdm import tqdm

In [5]:
# load pretrain resenet model
weights = ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 7)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 57.6MB/s]


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:
# Load the trained model and evaluate
model.load_state_dict(torch.load("/content/drive/MyDrive/ML/resnet_no_validate.pth"))

<ipython-input-7-0f5be5a424c0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/ML/resnet_no_validate.pth"))


<All keys matched successfully>

In [8]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of model parameters: {total_params}")

Total number of model parameters: 23522375


In [9]:
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [10]:
from PIL import Image
import glob

class TestingDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.names = []

        self.images = sorted(glob.glob(f"{self.img_dir}/*"))
        self.names = [os.path.basename(image)[:-4] for image in self.images]

    def __len__(self):
        return len(self.images)

    def __getnames__(self):
        return self.names

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("L")
        image = self.transform(image)
        return image

In [11]:
batch_size = 32
test_data = TestingDataset(f"{unzip_path}/data/Images/test", test_transform)
print(f"Total testing data: {test_data.__len__()}")
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Total testing data: 3589


In [13]:
import pandas as pd

model.eval()
predictions = []
with torch.no_grad():
  for i, images in enumerate(tqdm(test_loader)):
    images = images.to(device)
    logits = model(images)
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

# 預設的 index_mapping
index_mapping = {"Angry": 0, "Disgust": 1, "Fear": 2, "Happy": 3, "Neutral": 4, "Sad": 5, "Surprise": 6}

# 將預測索引轉換為對應類別名稱
class_dic = {v: k for k, v in index_mapping.items()}
predictions = [class_dic[pred] for pred in predictions]  # 索引 -> 類別名稱

# 模擬測試資料的檔案名稱
filenames = test_data.__getnames__()

# 將類別名稱轉為指定的數字標籤
labels = [index_mapping[pred] for pred in predictions]

# 建立 DataFrame 並輸出到 CSV
submission = pd.DataFrame({"filename": filenames, "label": labels})
submission.to_csv("/content/drive/MyDrive/ML/resnet_no_validate.csv", index=False)

100%|██████████| 113/113 [00:04<00:00, 25.96it/s]
